##Install and Imports

In [10]:
!pip install flair

In [11]:
!pip install vaderSentiment-fr

In [12]:
!pip install requests

In [13]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 66.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [14]:
import requests
from pprint import pprint
import os
import pandas as pd
import numpy as np
from flair.data import Sentence
from flair.models import SequenceTagger
import json
import re
import random
from tqdm import tqdm

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stopWords = set(stopwords.words('french'))
stemmer = SnowballStemmer(language='french')

import spacy
nlp = spacy.load("fr_core_news_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
# mount my Google Drive on the VM

from google.colab import drive
drive.mount('/gdrive')

DIR_PROJECT = '/gdrive/MyDrive/Projet5ADD/Tous les Codes'
DIR_LEXICON = os.path.join(DIR_PROJECT,'Code Sentiment Analysis/French-NRC-EmoLex.txt')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


#Extracting important words and reducing text

In [16]:
stopWords.add("n’")
stopWords.add("l’")
stopWords.add("c’")

In [17]:
#takes a sentence as a uniq string, returns a list of words without the stopwords
def cleaning_sentence(sentence):
  clean =[]
  for token in nlp(sentence):
    if token.text not in stopWords:
        clean.append(token.text)
  return clean

In [18]:
#Supposé réduire les mots à leur racine marche approximativement
def racine(sentence):
    return [stemmer.stem(X.text) for X in nlp(sentence)]

In [19]:
#classifie par type grammatical
def gramm(sentence):
    return [(X, X.pos_) for X in nlp(sentence)]

In [20]:
#prends une phrase, vire les stopwords, renvoie la racine et le type des mots restant (note: si le mot est un nom propre, racine et type seront erronés)
def extract_racines(sentence):
  nl = nlp(sentence)
  clean =[]
  for token in nlp(sentence):
    if token.text not in stopWords:
        clean.append(token)
  #return [(stemmer.stem(X.text),X.pos_) for X in clean]
  return [(X.text,X.pos_) for X in clean]

In [21]:
#trie le resultat de extract_racines pour ne garder que les noms et adjectifs
def filter_words(list_words):
  ret = []
  for words in list_words:
    if words[1] in ['ADJ','PROPN','NOUN']:
       ret.append(words[0])
  return ret

In [22]:
#TEST:
test = "Mélodie raconte une histoire, qui a eu lieu en 2013, sur son chien, Rex. Elle essaye depuis des années de publier son livre Rainbow, en Europe, plus partiuclièrement en France. C'est vrai qu'elle adore son chien. Elle aime la France du fond du coeur. Mais elle cherche aussi à devenir une star comme Macron."
filter_words(extract_racines(test))

['Mélodie',
 'raconte',
 'histoire',
 'lieu',
 'chien',
 'Rex',
 'années',
 'publier',
 'livre',
 'Rainbow',
 'Europe',
 'France',
 'vrai',
 'chien',
 'France',
 'fond',
 'coeur',
 'star',
 'Macron']

#Web-Scrapping

In [23]:
payload = {
    'source': 'google_search',
    'query': 'Wissal El-Kourdi',
    'parse': True,
    'no_images': True,
    'start_page': 1,
    'pages': 5,
    'limit': 10,
}


# Get response.
response = requests.request(
    'POST',
    'https://realtime.oxylabs.io/v1/queries',
    auth=('wissal', 'Maha10301030'),
    json=payload,
)


if response.status_code != 200:
    print("Error - ", response.json())
    exit(-1)


pprint(response.json())

Error -  {'message': 'Too many requests. (Product Fixed (successful only)).'}
{'message': 'Too many requests. (Product Fixed (successful only)).'}


In [24]:
text_json = json.loads(response.text)

In [25]:
text_json.keys()

dict_keys(['message'])

In [26]:
json_results = text_json['results']
type(json_results)

KeyError: 'results'

In [ ]:
description = []
for url in json_results:
  for content in (url['content']['results']['organic']):
      description.append(content['desc'])
description

In [ ]:
text_to_NER = ' '.join(description)

In [ ]:
text_to_NER.replace(u'\xa0', u' ')

#Pre-trained NER model

From https://huggingface.co/flair/ner-english-ontonotes-large

In [27]:
tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

def Retrieve_NER(sentence_string):
    """
    Retrieve Named Entities from a given sentence using Flair NER.

    Parameters:
    - sentence_string (str): The input sentence.

    Returns:
    - dict: A dictionary containing lists of entities for each NER category.
    """
    sentence = Sentence(sentence_string)

    # Predict entities
    tagger.predict(sentence)

    # Initialize a dictionary for storing unique entities for each category
    predicted_map = {
        'CARDINAL': set(),
        'DATE': set(),
        'EVENT': set(),
        'FAC': set(),
        'GPE': set(),
        'LANGUAGE': set(),
        'LAW': set(),
        'LOC': set(),
        'MONEY': set(),
        'NORP': set(),
        'ORDINAL': set(),
        'ORG': set(),
        'PERCENT': set(),
        'PERSON': set(),
        'PRODUCT': set(),
        'QUANTITY': set(),
        'TIME': set(),
        'WORK_OF_ART': set()
    }

    # Populate the dictionary with unique entities
    for entity in sentence.get_spans('ner'):
        text = entity.text.lower()
        predicted_map[entity.tag].add(text)

    return predicted_map


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2024-02-06 16:17:12,201 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [28]:
text_to_NER= "Mélodie raconte une histoire, qui a eu lieu en 2013, sur son chien, Rex. Elle essaye depuis des années de publier son livre Rainbow, en Europe, plus partiuclièrement en France. C'est vrai qu'elle adore son chien. Elle aime la France du fond du coeur. Mais elle cherche aussi à devenir une star comme Macron."
NER_to_algebra = Retrieve_NER(text_to_NER)

In [29]:
NER_to_algebra

{'CARDINAL': set(),
 'DATE': {'2013', 'des années'},
 'EVENT': set(),
 'FAC': set(),
 'GPE': {'france'},
 'LANGUAGE': set(),
 'LAW': set(),
 'LOC': {'europe'},
 'MONEY': set(),
 'NORP': set(),
 'ORDINAL': set(),
 'ORG': set(),
 'PERCENT': set(),
 'PERSON': {'macron', 'mélodie', 'rex'},
 'PRODUCT': set(),
 'QUANTITY': set(),
 'TIME': set(),
 'WORK_OF_ART': {'rainbow'}}

#Sentiment Classification

In [30]:
# Load NRC Emotion Lexicon

# METHODE 1
#print('DIR_LEXICON =', DIR_LEXICON)
#lexicon = pd.read_csv(DIR_LEXICON, sep='\t')

# METHODE 2
url = 'https://drive.google.com/file/d/1NGU4J7mhlqdJplLuVpjQJdq5OD5QTYGd/view?usp=sharing'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id
# read the data
lexicon = pd.read_csv(read_url)

In [31]:
# Create an instance of the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to perform sentiment analysis using VADER
def get_sentiment(tweet):
    sentiment = analyzer.polarity_scores(tweet)
    compound_score = sentiment['compound']
    return compound_score

# Define a function to categorize sentiment based on VADER score
def categorize_sentiment(score):
    if score < 0:
        return 'negative'
    elif score > 0:
        return 'positive'
    else:
        return 'neutral'

# Define a function to perform emotion detection using the lexicon
def detect_emotion(tweet):

    emotions = {'anger': 0, 'anticipation': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0, 'surprise': 0, 'trust': 0}

    words = tweet.lower().split()
    emotion_col = set(emotions.keys())
    for word in words:
        matches = lexicon[(lexicon['French Word'] == word)]
        if not matches.empty:
          for emotion in emotion_col:
            emotions[emotion] += matches[emotion].iloc[0]

    # Check if all emotion values are still zero
    if all(value == 0 for value in emotions.values()):
        return None

    # Determine the predominant emotion
    predominant_emotion = max(emotions, key=emotions.get)

    return predominant_emotion

In [32]:
#Returns a list of positive sentences from a list of sentences
def from_text_to_positive_sentences(list_sentences):
  positive_sentences = []
  for sentence in list_sentences:
    if get_sentiment(sentence)>0:
      positive_sentences.append(sentence)
  return positive_sentences

In [33]:

# Example sentence
example_sentence = "Fais chier, il pleut. Hier j'ai acheté une baguette. Elle était bonne. Sacré journée que nous avons eu là ! J'aime beaucoup les croissants. Je souffre jour après jour, mais on fait avec. Mes rhumatismes ne vont pas mieux mais merci de t'en soucier. Va crever ! Je suis allé courir ce matin. Il fait tellement chaud, on se croirait au printemps. "
example_sentence2 = "Mélodie raconte une histoire, qui a eu lieu en 2013, sur son chien, Rex. Elle essaye depuis des années de publier son livre Rainbow, en Europe, plus partiuclièrement en France. C'est vrai qu'elle adore son chien. Elle aime la France du fond du coeur. Mais elle cherche aussi à devenir une star comme Macron."
list_sentences = tokenize.sent_tokenize(example_sentence)
list_sentences2 = tokenize.sent_tokenize(example_sentence2)

In [34]:
from_text_to_positive_sentences(list_sentences)
#from_text_to_positive_sentences(list_sentences2)

['Elle était bonne.', "J'aime beaucoup les croissants."]

#Assembling the filtering parts

INPUT: FIRST PROFILE (insta for example)

In [35]:
input = "Georges est un homme de taille moyenne, de poids moyen, de visage banal, et à vrai dire, il ne présente aucun signe distinctif qui justifierais qu’on l’évoque ici, si ce n’est que d’après le commissaire, il était charmant. Georges est camionneur de profession, mais avant tout de passion, passion qui l’a même poussé à arrêter ses études de mathématiques, lorsqu’il a découvert qu’un mathématicien était rarement amené à conduire des camions. Il aime également beaucoup les chiffres et les choses symétriques, comme nous avons pu le constater précédemment. Cette passion l’a même poussé par le passé à acheter des choses dont il n’avait absolument pas besoin, juste pour satisfaire la symétrie de son appartement. Georges vit donc dans un appartement, modeste loft positionné au plein centre de Toulouse, hérité de ses parents, décédés d’un accident de chasse il y a de cela des années. Pour ce qui est de l’héritage, Georges n’aura pas eu à se battre, étant fils unique, et voilà donc pourquoi il habite là, malgré un salaire de camionneur qui ne lui permettrais pas d’acheter pareil endroit. Georges est un homme de goût, et en tant qu’homme de goût, il aime manger son pâté de campagne sur un pain de mie Harrys (celui aux céréales). C’est en revenant de ses courses que l’accident est arrivé. L’accident, en lui-même, n’avait que peu d’importance et n’allait pas impacter la vie de Georges outre mesure, si ce n’est pour le doux souvenir d’avoir mangé un excellent aligot."

In [36]:
def occurrences_NER(list_sentences):
    rows = []
    NER_glob = pd.DataFrame(columns=['CARDINAL','DATE','EVENT','FAC','GPE','LANGUAGE','LAW','LOC','MONEY','NORP','ORDINAL','ORG','PERCENT','PERSON','PRODUCT','QUANTITY','TIME','WORK_OF_ART'])
    for sentence in list_sentences:
        NER = Retrieve_NER(sentence)
        for key, word_list in NER.items():
            for word in word_list:
                new_row = pd.Series(index=NER_glob.columns)
                new_row[key] = word
                rows.append(new_row)
    NER_glob = pd.DataFrame(rows, columns=NER_glob.columns)
    return NER_glob

In [37]:
#First decompose in sentences
list_sentences = tokenize.sent_tokenize(input)

#Second NER directly

NER_neutral = occurrences_NER(list_sentences)

#Third Positive processing, only keep positive sentences
list_positive_sentences = from_text_to_positive_sentences(list_sentences)

#Fourth NLP process to get nouns, adj
list_words_NLP =[]
for sentence in list_positive_sentences:
  list_words_NLP.append(filter_words(extract_racines(sentence)))

#Five second NER on the positive sentences
NER_positive = occurrences_NER(list_positive_sentences)

#Add every result in a biiig dataframe and concacenate similar occurences, count them
NER_tot = pd.concat([NER_neutral, NER_positive], ignore_index=True)

#We have the dataframe NER_tot of words by NER category and the list_words_NLP of words

<ipython-input-36-0a2d129cb81c>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_row = pd.Series(index=NER_glob.columns)
<ipython-input-36-0a2d129cb81c>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_row = pd.Series(index=NER_glob.columns)


In [38]:
#here we do a quick data squeezing to add every words from both sets in a new dataframe where there is also the number of occurences.

# Initialize df_words with word, occurrences, and NER_category columns
df_words = pd.DataFrame(columns=['word','NER_category','occurrences']) #TASNIM : added the ner_category
word_counts = pd.Series(list_words_NLP).value_counts()

# Step 1: Extract words and their occurrences from list_words_NLP
for word_list in list_words_NLP:
    word_list = [word.lower() for word in word_list]
    word_counts = pd.Series(word_list).value_counts()
    new_df = pd.DataFrame({'word': word_counts.index, 'occurrences': word_counts.values})
    new_df['NER_category'] = 'N/A'  #TASNIM :  Initialize NER category as 'N/A'
    df_words = pd.concat([df_words, new_df], ignore_index=True)

# TASNIM

# Step 2: Update NER category for words from NER results in NER_tot
for idx, row in NER_tot.iterrows():
    for col in NER_tot.columns[1:]:  # Exclude the first column 'NER TOT'
        word = row[col]
        if pd.notnull(word):
            word = word.lower()
            # Update NER category for the corresponding word in df_words
            df_words.loc[df_words['word'] == word, 'NER_category'] = col

# Step 3: Group by word and NER_category and sum the occurrences
df_words = df_words.groupby(['word', 'NER_category'], as_index=False)['occurrences'].sum()

# Step 4: Sort by occurrences in descending order
df_words = df_words.sort_values(by='occurrences', ascending=False)

# Step 5: Reset index
df_words.reset_index(drop=True, inplace=True)

df_words

# MERLIN

#word_counts = NER_tot.apply(lambda row: row.dropna().iloc[0], axis=1).value_counts()
#new_df = pd.DataFrame({'word': word_counts.index, 'occurrences': word_counts.values})
#df_words = pd.concat([df_words, new_df.groupby('word')['occurrences'].sum().reset_index()], ignore_index=True)
#sum duplicates
#df_words = df_words.groupby('word', as_index=False)['occurrences'].sum()
#df_words = df_words.sort_values(by='occurrences', ascending=False)
#df_words

,word,NER_category,occurrences
0,georges,PERSON,5
1,homme,N/A,3
2,passion,N/A,3
3,camionneur,N/A,2
4,goût,N/A,2
5,choses,N/A,2
6,accident,N/A,1
7,poids,N/A,1
8,mie,N/A,1
9,moyen,N/A,1


End of here we have two main list of words:
*   df_words, words classified by number of occurrences in the combined sum of NER, and after positive combination NLP
* NER_tot ou NER_neutral/NER_positive: NER dataframes

INPUT SECOND PROFILE (fb for ex)

In [ ]:
input = 'blablabala'

INPUT BOTH PROFILES

In [ ]:
input = 'blablabala'

#Combination and Algebra







> Main Functions
    > - FILTER : NER dictionary → GPE, LOC, PERSON, CARDINAL, DATE
    > - SORT : according to the num of occurrences
    > - BUILD : the String Dictionary
    > - BUILD : the Digit Dictionary
    > - BUILD : the Special Dictionary
    > - Compose according to minimal requirement of most websites :
        - min 8 characters
        - string special digits
        - string digits special

> String Categories :
>
> - Name (masculine, feminine, animal) : Person_NER
> - Article : {I, my, the, it, we, you …}
> - City : GPE_NER , Location_NER
> - Keyboard : {qwerty, qwe, abc, asd,..}
> - Prepositions : {to, in …}

> Digit Categories
>
> - Number : Cardianl_NER , {0..9}
> - Common-number : {123456, 123, 123456789, 12345,1234, 11, 13, 12345678, 01, 10}
> - Year : Date_NER (filter only the year)

> Specials Categories
>
> - Simple : { . _   !   @   -   :   #    *   $   space   &   +   ?   ,   / }
> - Combined : {!!   .:   &#   **   …   :,   $$  __ }

In [39]:
from collections import Counter

#Filter and Sort Tags
def sort_and_filter_tags(output_NER, allowed_tags):
  #FILTER
  filtered = {tag: output_NER[tag] for tag in allowed_tags}

  #SORT
  for tag, values in filtered.items():
    if values:
        # Count occurrences of each element
        element_counts = Counter(values)

        # Sort elements based on their counts (in descending order)
        sorted = [element for element, count in element_counts.most_common(10)]

        # Replace the original list with the sorted list
        filtered[tag] = sorted

  return filtered

allowed_tags = ["GPE", "LOC", "PERSON", "CARDINAL", "DATE"]

filtered_NER_output = sort_and_filter_tags(NER_to_algebra, allowed_tags)

print("Filtered NER Tags:", filtered_NER_output)

Filtered NER Tags: {'GPE': ['france'], 'LOC': ['europe'], 'PERSON': ['mélodie', 'macron', 'rex'], 'CARDINAL': set(), 'DATE': ['des années', '2013']}


Build Dictionaries and Lists

In [40]:
import itertools

#Create dictionaries
string_dict = {
    'Name': filtered_NER_output.get('PERSON', []),
    'Article': ['I', 'my', 'the', 'it', 'we', 'you'],
    'City': filtered_NER_output.get('GPE', []) + filtered_NER_output.get('LOC', []),
    'Keyboard': ['qwerty', 'qwe', 'abc', 'asd'],
    'Prepositions': ['to', 'in'],
}
digit_dict = {
    'Number': list(filtered_NER_output.get('CARDINAL', [])) + [str(i) for i in range(10)],
    'Common': ['123456', '123', '123456789', '12345', '1234', '11', '13', '12345678', '01', '10'],
    'Year': [date[-4:] for date in filtered_NER_output.get('DATE', []) if date[-4:].isdigit()],
}
special_dict = {
    'Simple': ['.', '_', '!', '@', '-', ':', '#', '*', '$', ' ', '&', '+', '?', ',', '/'],
    'Combined': ['!!', '.:', '&#', '**', '…', ':', '$$', '__'],
}

# Create lists for each dictionary
string_list = list(itertools.chain.from_iterable(string_dict.values()))
digit_list = list(itertools.chain.from_iterable(digit_dict.values()))
special_list = list(itertools.chain.from_iterable(special_dict.values()))

Generate List of Passwords

In [80]:
def generate_passwords(strings, digits, specials):
    all_combinations = []

    # Generate combinations of string + special + digits
    combo1 = itertools.product(strings, specials, digits)
    all_combinations.extend(''.join(comb) for comb in combo1)

    # Generate combinations of string + digits + special
    combo2 = itertools.product(strings, digits, specials)
    all_combinations.extend(''.join(comb) for comb in combo2)

    # Filter combinations with length >= 8
    valid_passwords = [password for password in all_combinations if len(password) >= 8]

    return valid_passwords

password_combinations = generate_passwords(string_list, digit_list, special_list)

print("Generated Password Combinations:")
i = 0
for password in password_combinations:
    print(i , ' : ' , password)
    i+=1



Streaming output truncated to the last 5000 lines.
2748  :  europe__01
2749  :  europe__10
2750  :  europe__2013
2751  :  qwerty.0
2752  :  qwerty.1
2753  :  qwerty.2
2754  :  qwerty.3
2755  :  qwerty.4
2756  :  qwerty.5
2757  :  qwerty.6
2758  :  qwerty.7
2759  :  qwerty.8
2760  :  qwerty.9
2761  :  qwerty.123456
2762  :  qwerty.123
2763  :  qwerty.123456789
2764  :  qwerty.12345
2765  :  qwerty.1234
2766  :  qwerty.11
2767  :  qwerty.13
2768  :  qwerty.12345678
2769  :  qwerty.01
2770  :  qwerty.10
2771  :  qwerty.2013
2772  :  qwerty_0
2773  :  qwerty_1
2774  :  qwerty_2
2775  :  qwerty_3
2776  :  qwerty_4
2777  :  qwerty_5
2778  :  qwerty_6
2779  :  qwerty_7
2780  :  qwerty_8
2781  :  qwerty_9
2782  :  qwerty_123456
2783  :  qwerty_123
2784  :  qwerty_123456789
2785  :  qwerty_12345
2786  :  qwerty_1234
2787  :  qwerty_11
2788  :  qwerty_13
2789  :  qwerty_12345678
2790  :  qwerty_01
2791  :  qwerty_10
2792  :  qwerty_2013
2793  :  qwerty!0
2794  :  qwerty!1
2795  :  qwerty!2
2796 

Front
**bold text**

In [81]:
!pip install flask-ngrok

In [82]:
!pip install flask-bootstrap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460118 sha256=57cd5abbc17f230cb4cef7a10b1aa307b924769e3941f0e0abcbb91c5e03c7bd
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3926 sha256=1d0eca856e88e882587acef27f9f6177e20bc0082710d4ee3aea25b6c8da9e2d
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask-bootstrap visitor


In [88]:
from flask import Flask, render_template
from flask_bootstrap import Bootstrap
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
bootstrap = Bootstrap(app)
run_with_ngrok(app, port=5008)
@app.route("/")
def home():
  return render_template('front.html')

app.run(port=5008)

TypeError: run_with_ngrok() got an unexpected keyword argument 'port'

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask
from pyngrok import ngrok
import threading
import itertools

app = Flask(__name__)

def generate_passwords(strings, digits, specials):
    all_combinations = []

    # Generate combinations of string + special + digits
    combo1 = itertools.product(strings, specials, digits)
    all_combinations.extend(''.join(comb) for comb in combo1)

    # Generate combinations of string + digits + special
    combo2 = itertools.product(strings, digits, specials)
    all_combinations.extend(''.join(comb) for comb in combo2)

    # Filter combinations with length >= 8
    valid_passwords = [password for password in all_combinations if len(password) >= 8]

    return valid_passwords

@app.route('/generate_passwords', methods=['POST'])
def generate_passwords_route():
    string_list = ['abc', 'def']  # Example strings
    digit_list = ['123', '456']   # Example digits
    special_list = ['!@#', '$%^'] # Example specials

    password_combinations = generate_passwords(string_list, digit_list, special_list)

    # Generate output HTML
    output_html = "<h3>Generated Password Combinations:</h3>"
    for i, password in enumerate(password_combinations):
        output_html += f"<p>{i} : {password}</p>"

    return output_html

def start_ngrok():
    # Connect ngrok tunnel after Flask app starts running
    public_url = ngrok.connect(port=8000)
    print('Ngrok Tunnel URL:', public_url)

if __name__ == '__main__':
    # Start Flask application
    app_thread = threading.Thread(target=app.run, kwargs={'debug': True, 'port': 8000})
    app_thread.start()

    # Start ngrok in a separate thread
    ngrok_thread = threading.Thread(target=start_ngrok)
    ngrok_thread.start()


link back to front end

In [41]:
with open('front.html', 'r') as file:
  html_content = file.read()

In [50]:
from IPython.display import HTML , display
# Define your password generation function
def generate_passwords(strings, digits, specials):
    all_combinations = []

    # Generate combinations of string + special + digits
    combo1 = itertools.product(strings, specials, digits)
    all_combinations.extend(''.join(comb) for comb in combo1)

    # Generate combinations of string + digits + special
    combo2 = itertools.product(strings, digits, specials)
    all_combinations.extend(''.join(comb) for comb in combo2)

    # Filter combinations with length >= 8
    valid_passwords = [password for password in all_combinations if len(password) >= 8]

    return valid_passwords

password_combinations = generate_passwords(string_list, digit_list, special_list)

print("Generated Password Combinations:")
i = 0
for password in password_combinations:
    #print(i , ' : ' , password)
    i+=1


# Define a function to handle form submission
def handle_form_submission(form_data):
    # Extract form data and generate passwords
    nom = form_data['nom']
    prenom = form_data['prenom']
    date_of_birth = form_data['dateOfBirth']
    age = form_data['age']
    instagram_link = form_data['instagramLink']
    facebook_link = form_data['facebookLink']

    # Generate passwords based on the form data
    password_combinations = generate_passwords(string_list, digit_list, special_list)

    # Prepare the output as a string
    output = "<h3>Generated Password Combinations:</h3>"
    for i, password in enumerate(password_combinations):
        output += f"<p>{i}: {password}</p>"

    # Display output
    display(HTML(output))

# Register the handle_form_submission function to handle form submission
import google.colab.output
google.colab.output.register_callback('handle_form_submission', handle_form_submission)



Generated Password Combinations:


In [ ]:
from IPython.display import HTML
HTML(html_content)

In [55]:
!pip install pyngrok

In [57]:
ngrok.set_auth_token("2c059mxhNS9sb7rjYYcW350H8kj_7oA165Gpw8HVJW3fiFGDZ")

In [79]:
import requests

# Replace this URL with the ngrok tunnel URL or your local Flask server URL
flask_url = 'e1f4-104-196-104-240'  # Example URL, replace it with your actual URL

# Example POST request data
data = {
    'nom': 'John',
    'prenom': 'Doe',
    'dateOfBirth': '1990-01-01',
    'age': '30',
    'instagramLink': 'https://www.instagram.com/johndoe',
    'facebookLink': 'https://www.facebook.com/johndoe'
}
# Connect ngrok to the Flask app
# Connect ngrok to the Flask app with a random subdomain
tunnel = ngrok.connect(proto="http", name="my_tunnel")

# Extract the subdomain from the public URL of the tunnel
public_url = tunnel.public_url
subdomain = public_url.split('.')[0]

print('Ngrok Subdomain:', subdomain)


# Send a POST request to your Flask app
response = requests.post( 'https://' + flask_url + '/generate_passwords', data=data)

# Print the response from the server
print(response.text)


Ngrok Subdomain: https://1ce6-35-245-173-218


ConnectionError: HTTPSConnectionPool(host='e1f4-104-196-104-240', port=443): Max retries exceeded with url: /generate_passwords (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7a60fac35480>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
pip install Flask

In [ ]:
from flask import Flask, render_template, request
import itertools

app = Flask(__name__)

def generate_passwords(strings, digits, specials):
    all_combinations = []

    # Generate combinations of string + special + digits
    combo1 = itertools.product(strings, specials, digits)
    all_combinations.extend(''.join(comb) for comb in combo1)

    # Generate combinations of string + digits + special
    combo2 = itertools.product(strings, digits, specials)
    all_combinations.extend(''.join(comb) for comb in combo2)

    # Filter combinations with length >= 8
    valid_passwords = [password for password in all_combinations if len(password) >= 8]

    return valid_passwords

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/generate_passwords', methods=['POST'])
def generate_passwords_route():
    string_list = ['abc', 'def']  # Example strings
    digit_list = ['123', '456']   # Example digits
    special_list = ['!@#', '$%^'] # Example specials

    password_combinations = generate_passwords(string_list, digit_list, special_list)

    # Generate output HTML
    output_html = "<h3>Generated Password Combinations:</h3>"
    i = 0
    for password in password_combinations:
        output_html += f"<p>{i} : {password}</p>"
        i += 1

    return output_html

if __name__ == '__main__':
    app.run(debug=True)


#Computation of Passwords

#Comparison of Hashes


**MD5 (Message Digest Algorithm 5):**
Produces a 128-bit hash value, typically represented as a 32-character hexadecimal number.
Considered insecure for password storage due to vulnerabilities.

**SHA-1 (Secure Hash Algorithm 1):**
Produces a 160-bit hash value, typically represented as a 40-character hexadecimal number.
Deprecated for security-sensitive applications due to vulnerabilities.

**SHA-256, SHA-384, and SHA-512 (Secure Hash Algorithms):**
Part of the SHA-2 family, producing hash values of 256, 384, and 512 bits, respectively.
Widely used and considered secure for password hashing.

**bcrypt:**
Adaptive hash function based on the Blowfish cipher.
Designed to be slow and computationally intensive to resist brute-force attacks.
Commonly used for password storage.

**scrypt:**
Key derivation function designed to be memory-intensive, making it resistant to certain types of hardware attacks.
Provides protection against both brute-force and rainbow table attacks.

**Argon2:**
Winner of the Password Hashing Competition (PHC).
Designed to be memory-hard and resistant to side-channel attacks.
Provides a high level of security and is recommended for password hashing.

#Hash model

**SHA-256**
SHA, or Secure Hash Algorithm, SHA is a hashing algorithm used in secure connections to prove the integrity and authenticity of a message to the recipient. The SHA algorithm is the default hashing algorithm defined in SSL certificates.

In [ ]:
from hashlib import sha256
h = sha256()
h.update(b'azerty123')
hash = h.hexdigest()
print(hash)


In [ ]:
pip install passlib[bcrypt]
pip install passlib[argon2]

**Bcrypt :**

Salt is a fixed-length cryptographically-strong random value that is added to the input of hash functions to create unique hashes for every input. A salt is added to make a password hash output unique even for users adopting common passwords.

In [ ]:
import bcrypt

password = b'passWord'

salt = bcrypt.gensalt()
hashed = bcrypt.hashpw(password,salt)

print(hashed)

if bcrypt.checkpw(password, hashed):
    print("match")
else:
    print("does not match")

**Scrypt :**

In [ ]:
from passlib.hash import scrypt
h = scrypt.hash("password")
print(h)
scrypt.verify("password", h)


**Argon2** :


In [ ]:
from passlib.hash import argon2
h = argon2.hash("password")
print(h)
argon2.verify("password", h)

#Conclusion